In [1]:
import numpy as np
import time

In [2]:
path = '/Users/hasan/Desktop/Anaconda/IR-Fold3/'
doc = open(path+'train_sorted.txt').read().split('\n')

print(len(doc))

719112


In [4]:
s_time = time.time()
dict = {i:[] for i in range(1, 137)}
dict['idx'] = []
dict['qids'] = []

for i in range(len(doc)-1):
    d_list = doc[i].split()
    dict['idx'].append(d_list[0])
    dict['qids'].append(d_list[1])
    for d in d_list[2:]:
        s = d.split(':')
        dict[int(s[0])].append(float(s[1]))
time.time() - s_time

75.46295094490051

In [5]:
# store min and max of each feature in order to use in validation and test data
s_time = time.time()
minMax = []
for i in range(1, 137):
    minMax.append([min(dict[i]), max(dict[i])])
time.time() - s_time

9.599010944366455

In [6]:
np.savetxt(path+'minMax.csv', minMax, delimiter=',')

In [30]:
def f(x):
    '''avoids scientific numbers like 1e-7'''
    return "{:.6f}".format(x)
f = np.vectorize(f)

In [8]:
s_time = time.time()
for i in range(1, 137):
    dict[i] = list(f((np.array(dict[i]) - min(dict[i]))/(max(dict[i])-min(dict[i]))))
time.time() - s_time

104.50257301330566

In [12]:
s_time = time.time()
with open(path+'train_sorted_normalized.txt', 'w') as f:
    for j in range(len(doc)-1):
        string = dict['idx'][j] + ' ' + dict['qids'][j] + ' '
        for i in range(1, 137):
            string += str(i) + ':' + str(dict[i][j]) + ' '
        
        f.write(string)
        f.write('\n')
f.close()
time.time() - s_time

109.33551788330078

# Function for normalizing data

In [32]:
def normalize(path, minMax, name_to_save_file):
    '''
    path = path to data: '/Users/hasan/Desktop/Anaconda/IR-Fold3/'+'vali or test'
    minMax = min and max of features of shape (n_features, 2), i.e. 
             n_features x [min, max] which is calculated from train data
    name_to_save_file: a string like 'test_normalized.txt'
    '''
    s_time = time.time()
    doc = open(path).read().split('\n')
    
    dict = {i:[] for i in range(1, 137)}
    dict['idx'] = []
    dict['qids'] = []

    for i in range(len(doc)-1):
        d_list = doc[i].split()
        dict['idx'].append(d_list[0])
        dict['qids'].append(d_list[1])
        for d in d_list[2:]:
            s = d.split(':')
            dict[int(s[0])].append(float(s[1]))

    for i in range(1, 137):
        dict[i] = list(f((np.array(dict[i]) - minMax[i-1][0])/(minMax[i-1][1]- minMax[i-1][0])))
        
    with open(path+name_to_save_file, 'w') as g:
        for j in range(len(doc)-1):
            string = dict['idx'][j] + ' ' + dict['qids'][j] + ' '
            for i in range(1, 137):
                string += str(i) + ':' + str(dict[i][j]) + ' '

            g.write(string)
            g.write('\n')
    g.close()
    print('total time:', time.time() - s_time)


## Normalizing validation data

In [33]:
normalize(path=path+'vali.txt', minMax=minMax, name_to_save_file='vali_normalized.txt')

total time: 77.20620393753052


## Normalizing test data

In [34]:
normalize(path=path+'test.txt', minMax=minMax, name_to_save_file='test_normalized.txt')

total time: 70.11496019363403
